In [2]:
from nipype.interfaces.io import DataSink, SelectFiles, DataGrabber 
from nipype.interfaces.utility import IdentityInterface, Function    
from nipype.pipeline.engine import Node, Workflow, JoinNode, MapNode
from nipype.interfaces import fsl
from nipype import config, logging
from datetime import date
from pandas import Series, read_csv, to_numeric
from os import getcwd
from os.path import join, expanduser

In [53]:
# Set variables
user = expanduser('~')

#Set user for laptop
if user == '/Users/lucindasisk':
    base = '/Users/lucindasisk/Desktop/Milgram/candlab'
    home = join(base,'analyses/shapes/dwi')
    data_dir = join(base, 'analyses/shapes/dwi/eddyCUDA_data')
    workflow_dir = join(base, 'analyses/shapes/dwi/eddyCUDA_workflow')
    raw_dir = join(base, 'data/mri/bids_recon/shapes')

#Set user for Grace
if user == '/home/fas/gee_dylan/lms233':
    home = getcwd() + '/..'
    data_dir = join(home, 'eddyCUDA_data')
    workflow_dir = join(home, 'eddyCUDA_workflow')

#Set user for Milgram
if user == '/home/lms233':
    home = '/gpfs/milgram/project/gee_dylan/candlab'
    data_dir = 'analyses/shapes/dwi/data'
    workflow_dir = join(home, 'analyses/shapes/dwi/workflows')
    raw_dir = join(home, 'data/mri/bids_recon/shapes')
    
# Manual subject list
#subject_list = ['sub-A200', 'sub-A201', 'sub-A687', 'sub-A694', 'sub-A695', 'sub-A698']  # , 'sub-A201']
    
# Read in subject subject_list
subject_csv = read_csv(home + '/scripts/shapes/mri/dwi/shapes_dwi_subjList_08.07.2019.txt', sep=' ', header=None)
subject_list = subject_csv[0].values.tolist()

In [54]:
# Setup Datasink, Infosource, Selectfiles

datasink = Node(DataSink(base_directory=data_dir,
                         substitutions=[('_subject_id_', '')]),
                name='datasink')

# Set infosource iterables
infosource = Node(IdentityInterface(fields=['subject_id']),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list)]

# SelectFiles
template = dict(mask=join(home, 'analyses/shapes/dwi/data/2_Preprocessed/{subject_id}/{subject_id}_ses-shapesV1_T1w_flirt_brain_mask.nii.gz'),
                dti=join(
                    home, 'analyses/shapes/dwi/data/2_Preprocessed/{subject_id}/preprocessed_dwi.nii.gz'),
                bval=join(
                    raw_dir, '{subject_id}/ses-shapesV1/dwi/{subject_id}_ses-shapesV1_dwi.bval'),
                bvec=join(
                    raw_dir, '{subject_id}/ses-shapesV1/dwi/{subject_id}_ses-shapesV1_dwi.bvec'),
                aps=join(home, 'analyses/shapes/dwi/shapes_acqparams.txt'),
                index=join(home, 'analyses/shapes/dwi/shapes_index.txt')    
                )


sf = Node(SelectFiles(template,
                      base_directory=home),
          name='sf')


In [38]:
#Eddy_CUDA Node
# FSL Eddy correction to remove eddy current distortion

eddy = Node(fsl.Eddy(is_shelled=True,
                     interp='trilinear',
                     method='jac',
                     output_type='NIFTI_GZ',
                     residuals=True,
                     use_cuda=True,
                     cnr_maps=True,
                     repol=True),
            name='eddy')


In [39]:
eddy_flow = Workflow(name='eddy_flow')
eddy_flow.connect([(infosource, sf, [('subject_id', 'subject_id')]),
                   (sf, eddy, [('dti', 'in_file'),
                               ('bval', 'in_bval'),
                               ('bvec', 'in_bvec'),
                               ('index', 'in_index'),
                               ('aps', 'in_acqp'),
                               ('mask', 'in_mask')]),
                   (eddy, datasink, [('out_corrected', '2_Preprocessed'),
                                     ('out_rotated_bvecs', '2_Preprocessed.@par'),
                                     ('out_movement_rms',
                                      '2_Preprocessed.@par.@par'),
                                     ('out_parameter',
                                      '2_Preprocessed.@par.@par.@par'),
                                     ('out_restricted_movement_rms',
                                      '2_Preprocessed.@par.@par.@par.@par'),
                                     ('out_shell_alignment_parameters',
                                      '2_Preprocessed.@par.@par.@par.@par.@par'),
                                     ('out_cnr_maps',
                                      '2_Preprocessed.@par.@par.@par.@par.@par.@par'),
                                     ('out_residuals',
                                      '2_Preprocessed.@par.@par.@par.@par.@par.@par.@par'),
                                     ('out_outlier_report',
                                      '2_Preprocessed.@par.@par.@par.@par.@par.@par.@par.@par')])
                   ])
eddy_flow.base_dir = workflow_dir
eddy_flow.write_graph(graph2use='flat')
eddy = eddy_flow.run('MultiProc', plugin_args={'n_procs': 4})

/usr/local/anaconda3/lib/python3.6/site-packages/nipype/utils/misc.py:367: UserWarning: Current folder does not exist, replacing with "/Users/lucindasisk/Desktop/Milgram/candlab/scripts/shapes/mri/dwi/Diffusion" instead.
  warn('Current folder does not exist, replacing with "%s" instead.' % cwd)


190812-13:49:09,497 nipype.workflow INFO:
	 Generated workflow graph: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/eddyCUDA_workflow/eddy_flow/graph.png (graph2use=flat, simple_form=True).
190812-13:49:09,512 nipype.workflow INFO:
	 Workflow eddy_flow settings: ['check', 'execution', 'logging', 'monitoring']
190812-13:49:09,888 nipype.workflow INFO:
	 Running in parallel.
190812-13:49:09,893 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 6 jobs ready. Free memory (GB): 28.80/28.80, Free processors: 4/4.
190812-13:49:09,966 nipype.workflow INFO:
	 [Node] Outdated cache found for "eddy_flow.sf".
190812-13:49:09,987 nipype.workflow INFO:
	 [Node] Setting-up "eddy_flow.sf" in "/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/eddyCUDA_workflow/eddy_flow/_subject_id_sub-A698/sf".
190812-13:49:10,4 nipype.workflow INFO:
	 [Node] Outdated cache found for "eddy_flow.sf".
190812-13:49:10,22 nipype.workflow INFO:
	 [Node] Outdated cache found for "eddy_fl

Process Process-43:
Process Process-42:
Process Process-41:
Process Process-44:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/anaconda3/lib/python3.6/mu

KeyboardInterrupt: 

In [55]:
#eddyqc nodes

# eddyquad = Node(fsl.EddyQuad(),     
#                name='eddyquad')

In [56]:
# Workflow

# eddyqc_flow = Workflow(name='eddyqc_flow')
# eddyqc_flow.connect([(infosource, sf, [('subject_id', 'subject_id')]),
#                      (sf, eddyquad, [('eddy_base','base_name'),
#                                      ('index', 'idx_file'),
#                                      ('aps', 'param_file'),
#                                      ('mask', 'mask_file'),
#                                      ('bval', 'bval_file'),
#                                      ('bvec','bvec_file')])
#                    ])
# eddyqc_flow.base_dir = workflow_dir
# eddyqc_flow.write_graph(graph2use='flat')
# eddyqc = eddyqc_flow.run('MultiProc', plugin_args={'n_procs': 4})


190812-13:53:45,239 nipype.workflow INFO:
	 Generated workflow graph: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/eddyCUDA_workflow/eddyqc_flow/graph.png (graph2use=flat, simple_form=True).
190812-13:53:45,252 nipype.workflow INFO:
	 Workflow eddyqc_flow settings: ['check', 'execution', 'logging', 'monitoring']
190812-13:53:45,505 nipype.workflow INFO:
	 Running in parallel.
190812-13:53:45,508 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 6 jobs ready. Free memory (GB): 28.80/28.80, Free processors: 4/4.
190812-13:53:45,588 nipype.workflow INFO:
	 [Node] Setting-up "eddyqc_flow.sf" in "/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/eddyCUDA_workflow/eddyqc_flow/_subject_id_sub-A698/sf".
190812-13:53:45,621 nipype.workflow INFO:
	 [Node] Setting-up "eddyqc_flow.sf" in "/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/eddyCUDA_workflow/eddyqc_flow/_subject_id_sub-A695/sf".
190812-13:53:45,652 nipype.workflow INFO:
	 [Node] Sett

190812-13:53:47,711 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 2 jobs ready. Free memory (GB): 28.80/28.80, Free processors: 4/4.
190812-13:53:47,781 nipype.workflow INFO:
	 [Node] Setting-up "eddyqc_flow.sf" in "/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/eddyCUDA_workflow/eddyqc_flow/_subject_id_sub-A201/sf".
190812-13:53:47,820 nipype.workflow INFO:
	 [Node] Setting-up "eddyqc_flow.sf" in "/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/eddyCUDA_workflow/eddyqc_flow/_subject_id_sub-A200/sf".
190812-13:53:48,97 nipype.workflow INFO:
	 [Node] Running "sf" ("nipype.interfaces.io.SelectFiles")
190812-13:53:48,113 nipype.workflow INFO:
	 [Node] Running "sf" ("nipype.interfaces.io.SelectFiles")
190812-13:53:48,159 nipype.workflow WARNING:
	 [Node] Error on "eddyqc_flow.sf" (/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/eddyCUDA_workflow/eddyqc_flow/_subject_id_sub-A201/sf)
190812-13:53:48,179 nipype.workflow WARNING:
	 [Node

RuntimeError: Workflow did not execute cleanly. Check log for details